In [1]:
import pdfplumber
import pandas as pd
import re

In [2]:
pdf = pdfplumber.open("florida-warn-2018.pdf")

## Basic solution

In [3]:
all_rows = []
for page in pdf.pages:
    print(f"Parsing page {page.page_number}")
    all_rows += page.extract_table()
    
len(all_rows)

Parsing page 1
Parsing page 2
Parsing page 3
Parsing page 4
Parsing page 5
Parsing page 6
Parsing page 7
Parsing page 8
Parsing page 9
Parsing page 10
Parsing page 11
Parsing page 12
Parsing page 13
Parsing page 14
Parsing page 15
Parsing page 16


134

In [4]:
pd.DataFrame(all_rows[1:], columns=all_rows[0])

,COMPANY NAME,NOTICE\nDATE,LAYOFF\nDATE,EMPLOYEES\nAFFECTED,INDUSTRY
0,"Hotelbeds\n5422 Carrier Srive,\nSuite 201\nOrl...",12/19/2018,2/28/2019\nthru\n2/28/2019,16,Administrative and Support\nand Waste Manageme...
1,"Molina Healthcare\nof Florida, Inc.\n8300 N.W....",12/17/2018,2/12/2019\nthru\n2/12/2019,1,Health Care and Social\nAssistance
2,"First Transit, Inc.\nMiami International\nAirp...",12/13/2018,1/31/2019\nthru\n2/28/2019,85,Transportation and\nWarehousing
3,"H.D. Smith, LLC\n1901 N.W. 25th\nAvenue\nPompa...",12/13/2018,2/15/2019\nthru\n2/15/2019,20,Wholesale Trade
4,Bay Medical Sacred\nHeart Hospital\n615 N. Bon...,12/7/2018,2/4/2019\nthru\n2/4/2019,634,Health Care and Social\nAssistance
...,...,...,...,...,...
128,Kmart Store\n#04311\n1602 E. Brandon\nBlvd.\nB...,1/9/2018,4/1/2018\nthru\n4/15/2018,85,Retail Trade
129,"Brookdale Senior\nLiving, Inc.\n5600 Lakeside ...",1/8/2018,3/9/2018\nthru\n3/25/2018,110,Health Care and Social\nAssistance
130,Kmart Store #\n04420\n3711 E. Silver Spring\nB...,1/8/2018,4/8/2018\nthru\n4/22/2018,64,Retail Trade
131,Macy's Downtown\nMiami Store\n22 East Flagler ...,1/8/2018,3/18/2018\nthru\n3/31/2018,161,Retail Trade


## Solution with improvements

Note: I don't expect students to figure out how to do the name-splitting, as it's a bit more advanced than we covered in class.

- Separating the company names from the company addresses
- Normalizing the whitespace
- Fixing the entries that span pages

In [5]:
pd.DataFrame(pdf.pages[1].chars)["fontname"].value_counts()

fontname
TimesNewRomanPSMT           956
Arial-Black                 191
GAERMB+Calibri               61
TimesNewRomanPS-ItalicMT     32
Name: count, dtype: int64

In [6]:
def only_arial_black_chars(obj):
    if "fontname" in obj:
        return "Black" in obj["fontname"]
    else:
        return True

In [7]:
def normalize_whitespace(row):
    return [ re.sub(r"\s+", " ", cell).strip()
        for cell in row ]

In [8]:
all_rows = []
for page in pdf.pages:
    table = page.extract_table()
    arial_black = page.filter(only_arial_black_chars).extract_table()
    for i in range(len(table)):
        table[i] = normalize_whitespace(arial_black[i][:1] + table[i])
    all_rows += table

Note how a cell that spans two pages is represented:

In [9]:
all_rows[16:19]

[['C&S Wholesale Services, Inc.',
  'C&S Wholesale Services, Inc.',
  '11/13/2018',
  '1/23/2019 thru 3/15/2019',
  '205',
  'Wholesale Trade'],
 ['', '4401 Seaboard Road Orlando, FL 32808', '', '', '', ''],
 ['Service America Enterprise, Inc.',
  'Service America Enterprise, Inc. Statewide Statewide, FL 00000',
  '11/13/2018',
  '1/18/2019 thru 1/18/2019',
  '161',
  'Construction']]

Here's one way to fix this:

In [10]:
compact_rows = []
for row in all_rows:
    if row[3] == "":
        compact_rows[-1] = [ (compact_rows[-1][i] + " " + cell).strip()
            for i, cell in enumerate(row) ]
    else:
        compact_rows.append(row)
        
compact_rows[16:19]

[['C&S Wholesale Services, Inc.',
  'C&S Wholesale Services, Inc. 4401 Seaboard Road Orlando, FL 32808',
  '11/13/2018',
  '1/23/2019 thru 3/15/2019',
  '205',
  'Wholesale Trade'],
 ['Service America Enterprise, Inc.',
  'Service America Enterprise, Inc. Statewide Statewide, FL 00000',
  '11/13/2018',
  '1/18/2019 thru 1/18/2019',
  '161',
  'Construction'],
 ['Molina Healthcare of Florida, Inc.',
  'Molina Healthcare of Florida, Inc. Statewide Statewide, FL 00000',
  '11/6/2018',
  '1/2/2019 thru 1/2/2019',
  '126',
  'Health Care and Social Assistance']]

In [11]:
(
    pd.DataFrame(compact_rows[1:], columns=all_rows[0])
    .rename(columns = {
        "": "COMPANY NAME",
        "COMPANY NAME": "COMPANY ADDRESS"
    })
)

,COMPANY NAME,COMPANY ADDRESS,NOTICE DATE,LAYOFF DATE,EMPLOYEES AFFECTED,INDUSTRY
0,Hotelbeds,"Hotelbeds 5422 Carrier Srive, Suite 201 Orland...",12/19/2018,2/28/2019 thru 2/28/2019,16,Administrative and Support and Waste Managemen...
1,"Molina Healthcare of Florida, Inc.","Molina Healthcare of Florida, Inc. 8300 N.W. 3...",12/17/2018,2/12/2019 thru 2/12/2019,1,Health Care and Social Assistance
2,"First Transit, Inc.","First Transit, Inc. Miami International Airpor...",12/13/2018,1/31/2019 thru 2/28/2019,85,Transportation and Warehousing
3,"H.D. Smith, LLC","H.D. Smith, LLC 1901 N.W. 25th Avenue Pompano ...",12/13/2018,2/15/2019 thru 2/15/2019,20,Wholesale Trade
4,Bay Medical Sacred Heart Hospital,Bay Medical Sacred Heart Hospital 615 N. Bonit...,12/7/2018,2/4/2019 thru 2/4/2019,634,Health Care and Social Assistance
...,...,...,...,...,...,...
120,Kmart Store #04311,Kmart Store #04311 1602 E. Brandon Blvd. Brand...,1/9/2018,4/1/2018 thru 4/15/2018,85,Retail Trade
121,"Brookdale Senior Living, Inc.","Brookdale Senior Living, Inc. 5600 Lakeside Dr...",1/8/2018,3/9/2018 thru 3/25/2018,110,Health Care and Social Assistance
122,Kmart Store # 04420,Kmart Store # 04420 3711 E. Silver Spring Blvd...,1/8/2018,4/8/2018 thru 4/22/2018,64,Retail Trade
123,Macy's Downtown Miami Store,Macy's Downtown Miami Store 22 East Flagler St...,1/8/2018,3/18/2018 thru 3/31/2018,161,Retail Trade


---

---

---